## Notebook Purpose
In this notebook I take two datasets: one which has information on countries (dataset 2) and their GDP and population, another which has information on Olympic athletes and their medals wins in the past 120 years by country (dataset 1). I processed the data such that I have new merged data frame on countries with athlete and medal count of countries in the olympics in the last 120 years along with their GDP and population

In [1]:
import pandas as pd
import numpy as np

In [54]:
# (dataset 1)
master = pd.read_csv("athlete_events.csv")
master.head(10)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN
9,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,"Speed Skating Women's 1,000 metres",NaN


In [12]:
master.shape

(271116, 15)

In [4]:
#(dataset 2)
countries = pd.read_csv("countries.csv")
countries.head(10)

,country,code,population,gdp_per_capita
0,Afghanistan,AFG,32526562.0,594.323081
1,Albania,ALB,2889167.0,3945.217582
2,Algeria,ALG,39666519.0,4206.031232
3,American Samoa*,ASA,55538.0,NaN
4,Andorra,AND,70473.0,NaN
5,Angola,ANG,25021974.0,4101.472152
6,Antigua and Barbuda,ANT,91818.0,13714.731962
7,Argentina,ARG,43416755.0,13431.878340
8,Armenia,ARM,3017712.0,3489.127690
9,Aruba*,ARU,103889.0,NaN


In [13]:
countries.shape

(201, 4)

In [11]:
# filtering for countries common in both datasets
master_c = master[master['Team'].isin(countries['country'])]
master_c.shape

(222254, 15)

In [14]:
# filling NA medal values with string
master_c['Medal'] = master_c['Medal'].fillna("Nothing")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
# getting the athlete count per country
athlete_count = master_c['Team'].value_counts().to_frame(name = 'count').reset_index()
athlete_count.sort_values(by=['index'], inplace=True, ascending=True)
athlete_count = athlete_count.reset_index(drop=True)
athlete_count.head(10)

,index,count
0,Afghanistan,126
1,Albania,70
2,Algeria,551
3,Andorra,169
4,Angola,267
5,Antigua and Barbuda,133
6,Argentina,3199
7,Armenia,221
8,Australia,7513
9,Austria,4866


In [40]:
# getting the different medals count per country
medals = master_c.groupby('Team')['Medal'].value_counts().to_frame(name = 'count').reset_index()
medals = medals.pivot_table('count', ['Team'], 'Medal').fillna(0)
medals = medals.rename(columns = {'Medal':'country'})
s = pd.Series(range(medals.shape[0]))
medals['country'] = medals.index.values
medals = medals.set_index([s])
medals.head(10)

Medal,Bronze,Gold,Nothing,Silver,country
0,2.0,0.0,124.0,0.0,Afghanistan
1,0.0,0.0,70.0,0.0,Albania
2,8.0,5.0,534.0,4.0,Algeria
3,0.0,0.0,169.0,0.0,Andorra
4,0.0,0.0,267.0,0.0,Angola
5,0.0,0.0,133.0,0.0,Antigua and Barbuda
6,91.0,91.0,2933.0,84.0,Argentina
7,9.0,2.0,205.0,5.0,Armenia
8,511.0,342.0,6207.0,453.0,Australia
9,150.0,95.0,4453.0,168.0,Austria


In [41]:
# merging the two datasets
country_medals= pd.merge(left=countries, right=medals, on="country")
country_medals.head(10)

,country,code,population,gdp_per_capita,Bronze,Gold,Nothing,Silver
0,Afghanistan,AFG,32526562.0,594.323081,2.0,0.0,124.0,0.0
1,Albania,ALB,2889167.0,3945.217582,0.0,0.0,70.0,0.0
2,Algeria,ALG,39666519.0,4206.031232,8.0,5.0,534.0,4.0
3,Andorra,AND,70473.0,NaN,0.0,0.0,169.0,0.0
4,Angola,ANG,25021974.0,4101.472152,0.0,0.0,267.0,0.0
5,Antigua and Barbuda,ANT,91818.0,13714.731962,0.0,0.0,133.0,0.0
6,Argentina,ARG,43416755.0,13431.878340,91.0,91.0,2933.0,84.0
7,Armenia,ARM,3017712.0,3489.127690,9.0,2.0,205.0,5.0
8,Australia,AUS,23781169.0,56310.962993,511.0,342.0,6207.0,453.0
9,Austria,AUT,8611088.0,43774.985174,150.0,95.0,4453.0,168.0


In [52]:
# final dataset
country_medals['athletes'] = athlete_count['count']
country_medals.head(10)

,country,code,population,gdp_per_capita,Bronze,Gold,Nothing,Silver,athletes
0,Afghanistan,AFG,32526562.0,594.323081,2.0,0.0,124.0,0.0,126
1,Albania,ALB,2889167.0,3945.217582,0.0,0.0,70.0,0.0,70
2,Algeria,ALG,39666519.0,4206.031232,8.0,5.0,534.0,4.0,551
3,Andorra,AND,70473.0,NaN,0.0,0.0,169.0,0.0,169
4,Angola,ANG,25021974.0,4101.472152,0.0,0.0,267.0,0.0,267
5,Antigua and Barbuda,ANT,91818.0,13714.731962,0.0,0.0,133.0,0.0,133
6,Argentina,ARG,43416755.0,13431.878340,91.0,91.0,2933.0,84.0,3199
7,Armenia,ARM,3017712.0,3489.127690,9.0,2.0,205.0,5.0,221
8,Australia,AUS,23781169.0,56310.962993,511.0,342.0,6207.0,453.0,7513
9,Austria,AUT,8611088.0,43774.985174,150.0,95.0,4453.0,168.0,4866


In [53]:
# saving to csv
country_medals.to_csv(r'country_extended.csv')